In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from underwriting import inventory_freshness_analyzer as ifa
from underwriting import inventory_freshness_analysis_query as ifa_query
import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_END_DATE = '2023-01-01'
today = date.today()

# Analyzer

In [4]:
analyzer = ifa.Analyzer()

In [5]:
# Updating the common attributes used for pulling query information
analyzer.update_class_attributes(['EL'], ['C9-0000370-LIC'], True)

In [6]:
# Find top ranked products/product categories by customized order column
analyzer.find_most_valuable_products('tx_product_name', 'avg_days_since_sale', 'ASC', 10)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price
0,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000
1,Kings Garden GMO Flower Prepackaged Quarter*,4.866667,10,0,15,16.0,95.000000
2,Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,130.000000
3,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000
4,XJ-13 1/4 oz Jar,6.812500,22,1,16,16.0,80.000000
5,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667
6,Flower Jars - Peanut Butter Breath - 3.5g,7.181818,15,0,22,28.0,42.795455
7,Wedding Cake LR Liquid Diamonds 1g 510 Vape Ca...,7.583333,15,1,12,12.0,45.000000
8,Kings Garden Cake Walk Live Budder (1g),8.250000,14,0,12,12.0,35.000000
9,HUMBOLDT TREES - Pre Roll Single - 0.7g - Tropaya,8.576923,17,1,26,32.0,9.961538


In [7]:
# Find top ranked product names categories by sale velocity (Without quantity normalization)
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_name', 15)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Surplus: 1g Cart: Blueberry Cookies [I],27.652174,59,5,138,148.0,39.913043,4.990566,5.352201
1,Surplus: 1g Cart: Trainwreck [H],32.372671,92,3,161,175.0,39.751553,4.973331,5.405794
2,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
3,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
4,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410
5,Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,130.000000,4.000000,4.000000
6,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,55.000000,3.701562,4.349335
7,Surplus: 1g Cart: Purple Lemonade [S],29.333333,70,3,108,125.0,40.000000,3.681818,4.261364
8,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
9,Calm 108-1g Pre-roll,8.870968,21,2,31,32.0,1.000000,3.494545,3.607273


In [8]:
# Find top ranked product names by sale velocity (With quantity normalization)
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_name', 15, True)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
1,Surplus: 1g Cart: Trainwreck [H],32.372671,92,3,161,175.0,39.751553,4.973331,5.405794
2,Surplus: 1g Cart: Blueberry Cookies [I],27.652174,59,5,138,148.0,39.913043,4.990566,5.352201
3,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
4,Surplus: 1g Cart: Green Crack [S],39.838235,87,4,136,198.0,39.161029,3.413806,4.970100
5,Kings Garden GMO Flower Prepackaged Gram*,10.041667,26,1,24,48.0,16.995833,2.390041,4.780083
6,Kings Garden Wedding Gushers Flower Prepackage...,14.225806,36,1,31,68.0,15.379032,2.179138,4.780045
7,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,55.000000,3.701562,4.349335
8,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
9,Surplus: 1g Cart: Purple Lemonade [S],29.333333,70,3,108,125.0,40.000000,3.681818,4.261364


In [9]:
# Find top ranked product category names categories by sale velocity (Without quantity normalization)
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15)

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Vape Cartridge (volume - each),Surplus: 1g Cart: Blueberry Cookies [I],27.652174,59,5,138,148.0,39.913043,4.990566,5.352201
1,Vape Cartridge (volume - each),Surplus: 1g Cart: Trainwreck [H],32.372671,92,3,161,175.0,39.751553,4.973331,5.405794
2,Pre-Roll Leaf,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
3,Pre-Roll Flower,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
4,Pre-Roll Flower,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410
5,Flower (packaged half ounce - each),Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,130.000000,4.000000,4.000000
6,Capsule (weight - each),"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,55.000000,3.701562,4.349335
7,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
8,Pre-Roll Flower,Calm 108-1g Pre-roll,8.870968,21,2,31,32.0,1.000000,3.494545,3.607273
9,Pre-Roll Flower,1.0g Cone - Mendo Breath,35.264463,169,1,121,125.0,6.113388,3.431216,3.544645


In [10]:
# Find top ranked product category names categories by sale velocity (With quantity normalization)
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15, True)

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Pre-Roll Flower,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
1,Vape Cartridge (volume - each),Surplus: 1g Cart: Trainwreck [H],32.372671,92,3,161,175.0,39.751553,4.973331,5.405794
2,Vape Cartridge (volume - each),Surplus: 1g Cart: Blueberry Cookies [I],27.652174,59,5,138,148.0,39.913043,4.990566,5.352201
3,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
4,Vape Cartridge (volume - each),Surplus: 1g Cart: Green Crack [S],39.838235,87,4,136,198.0,39.161029,3.413806,4.970100
5,Flower (packaged gram - each),Kings Garden GMO Flower Prepackaged Gram*,10.041667,26,1,24,48.0,16.995833,2.390041,4.780083
6,Flower (packaged gram - each),Kings Garden Wedding Gushers Flower Prepackage...,14.225806,36,1,31,68.0,15.379032,2.179138,4.780045
7,Capsule (weight - each),"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,55.000000,3.701562,4.349335
8,Pre-Roll Leaf,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
9,Pre-Roll Flower,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410


In [11]:
# Output data for inventory valuation calculation using Inventory-Sales joined data 
df_v1 = analyzer.get_inventory_valuation_query('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank
0,11497141,25.60,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1
...,...,...,...,...,...,...,...,...
281,8164573,21.00,0.0,BIIIT - Sour Straws Assorted,Edible (weight - each),0.320548,0.00,1
282,8164574,21.00,0.0,BIIIT - Sour Straws Assorted,Edible (weight - each),0.320548,0.00,1
283,8486461,23.04,4.0,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),1.405479,92.16,1
284,9166114,64.00,1.0,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.358904,64.00,1


In [12]:
# Output data for inventory valuation calculation using Inventory-Sales joined data 
df_v2 = analyzer.get_inventory_valuation_query('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit
0,26.420454,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333
...,...,...,...,...,...,...,...
281,21.505694,8164573,BIIIT - Sour Straws Assorted,Edible (weight - each),0.0,0.320548,22.800000
282,21.505694,8164574,BIIIT - Sour Straws Assorted,Edible (weight - each),0.0,0.320548,22.800000
283,27.095957,8486461,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),4.0,1.405479,35.010000
284,54.118077,9166114,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.0,1.358904,69.333333


In [13]:
# Output data and inventory valuation using Inventory-Sales joined data and last sale price method 
df_v1, valuation_1 = analyzer.find_inventory_valuation('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1.head()

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank,product_type,shelf_life,expired
0,11497141,0.00,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1,Concentrates,1.0,True
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1,Concentrates,1.0,False
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1,Edibles,1.0,False
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1,Concentrates,1.0,False
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1,Concentrates,1.0,False


In [14]:
valuation_1

95065.68

In [15]:
# Output data and inventory valuation using Inventory-Sales joined data and discount valuation method 
df_v2, valuation_2 = analyzer.find_inventory_valuation('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727,Concentrates,1.0,True
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333,Concentrates,1.0,False
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667,Edibles,1.0,False
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000,Concentrates,1.0,False
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333,Concentrates,1.0,False


In [17]:
valuation_2

93426.38496919227

In [18]:
# Output data and inventory valuation using Inventory-Incoming joined data and last sale price method 
df_i1, valuation = analyzer.find_incoming_inventory_valuation('discount_valuation', discount_rate=.2)
df_i1.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,15.0,Concentrates,1.0,True
1,11.140648,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,13.0,Concentrates,1.0,False
2,6.852360,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,8.0,Edibles,1.0,False
3,10.085278,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,12.0,Concentrates,1.0,False
4,10.120604,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,12.0,Concentrates,1.0,False


In [19]:
valuation

44311.11100094838

In [20]:
# Output data and inventory valuation using Inventory-Incoming joined data, joined on package_id and last sale price method 
df_i1, valuation = analyzer.find_incoming_inventory_valuation('discount_valuation', False, discount_rate=.2)
df_i1.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,15.0,Concentrates,1.0,True
1,11.140648,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,13.0,Concentrates,1.0,False
2,6.852360,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,8.0,Edibles,1.0,False
3,10.085278,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,12.0,Concentrates,1.0,False
4,10.120604,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,12.0,Concentrates,1.0,False


In [21]:
valuation

44576.099563279

In [23]:
# Find inventory valuation using all available methods using data that is joined by package_id
analyzer.find_all_valuation(False, discount_rate=.2)


#### Inventory Valuation based on sales data using last sale method: $85384.0 ####
#### Inventory Valuation based on sales data using discount time method: $83506.0 ####
#### Inventory Valuation based on incoming data using discount time method: $44576.0 ####


(85384.0, 83506.0, 44576.0)

In [24]:
# Find inventory valuation using all available methods using data that is joined by product_name
analyzer.find_all_valuation(True, discount_rate=.2)


#### Inventory Valuation based on sales data using last sale method: $95066.0 ####
#### Inventory Valuation based on sales data using discount time method: $93426.0 ####
#### Inventory Valuation based on incoming data using discount time method: $44311.0 ####


(95066.0, 93426.0, 44311.0)